In [1]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

## Get the Data

In [4]:
table='hackathon_02'
host='batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com'
port=5432
username='ldsa_student'
password='R4Fr4P3aAgMYBqqP'
dbname='s02_db'
schema='public'

In [7]:
conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, dbname)
conn_args = {'options': '-csearch_path={}'.format(schema)}

In [8]:
engine = create_engine(conn_str, connect_args=conn_args)

In [15]:
query= 'SELECT * FROM hackathon_02'
df_db_table_info = pd.read_sql_query(query, engine)
display(df_db_table_info)

,Timestamp,MA10,RSI30,RSI10,%K10
0,1325317920,NaN,NaN,NaN,NaN
1,1325346600,NaN,NaN,NaN,NaN
2,1325350740,NaN,NaN,NaN,NaN
3,1325350800,NaN,NaN,NaN,NaN
4,1325391360,NaN,NaN,NaN,NaN
...,...,...,...,...,...
19611,1351989660,10.632,53.396343,57.953487,100.000000
19612,1351991700,10.620,41.725604,26.388609,0.000000
19613,1351994460,10.632,50.897150,50.923396,82.608696
19614,1351994760,10.638,50.464714,49.949809,78.260870


We have a single table, named "missingdata". We'll be selecting everything from that table and saving it in a dataframe

In [16]:
def show_dtypes_na(df: pd.DataFrame) -> pd.DataFrame: 
    df_show = pd.concat(
        [
            df.dtypes.sort_index(), 
            df.isna().sum().sort_index()
        ],
        axis=1
    ).rename({0: 'dtypes', 1: 'na'}, axis=1)
    
    return df_show

In [22]:
'''query_missingdata = """
SELECT *
FROM hackathon_02
"""
df = pd.read_sql_query(query_missingdata, engine).drop(columns=['index'])
df.columns= df.columns.str.lower()

df'''

'query_missingdata = """\nSELECT *\nFROM hackathon_02\n"""\ndf = pd.read_sql_query(query_missingdata, engine).drop(columns=[\'index\'])\ndf.columns= df.columns.str.lower()\n\ndf'

In [27]:
query_missingdata = """
SELECT *
FROM hackathon_02
"""
df = pd.read_sql_query(query_missingdata, engine)
df.columns= df.columns.str.lower()

df

,timestamp,ma10,rsi30,rsi10,%k10
0,1325317920,NaN,NaN,NaN,NaN
1,1325346600,NaN,NaN,NaN,NaN
2,1325350740,NaN,NaN,NaN,NaN
3,1325350800,NaN,NaN,NaN,NaN
4,1325391360,NaN,NaN,NaN,NaN
...,...,...,...,...,...
19611,1351989660,10.632,53.396343,57.953487,100.000000
19612,1351991700,10.620,41.725604,26.388609,0.000000
19613,1351994460,10.632,50.897150,50.923396,82.608696
19614,1351994760,10.638,50.464714,49.949809,78.260870


In [24]:
show_dtypes_na(df)

,dtypes,na
%k10,float64,666
ma10,float64,9
rsi10,float64,10
rsi30,float64,30


In [28]:

df = df[pd.to_numeric(df['timestamp'], errors='coerce').notnull()].set_index('timestamp')
df.index = df.index.astype(int)
df

,ma10,rsi30,rsi10,%k10
timestamp,,,,
1325317920,NaN,NaN,NaN,NaN
1325346600,NaN,NaN,NaN,NaN
1325350740,NaN,NaN,NaN,NaN
1325350800,NaN,NaN,NaN,NaN
1325391360,NaN,NaN,NaN,NaN
...,...,...,...,...
1351989660,10.632,53.396343,57.953487,100.000000
1351991700,10.620,41.725604,26.388609,0.000000
1351994460,10.632,50.897150,50.923396,82.608696


In [29]:
df.to_csv('database_data.csv')